# Описание ноутбука
Содержит скрипты для перевода аудио из mp3 в wav. Wav в разреженные спектрограммы и формат npy. Затем разбивает npz файл песни на несколько npz файлов аккордов.

Т.о. на вход ноутбука подаются mp3 файлы, а на выходе получается список спектрограмм аккордов.

In [1]:
import pandas as pd
from pydub import AudioSegment
import os
import shutil
from tqdm import tqdm_notebook
import numpy as np
from scipy.sparse import csc_matrix,load_npz, save_npz, hstack
from scipy.io import wavfile
from scipy import signal, absolute
import stft

from chord_scripts import format_fname, string_to_secs, secs_to_string

In [2]:
BASE_DIR = os.path.realpath(os.getcwd()+'../../..')
DSETS_DIR = BASE_DIR+'/dsets/'
MP3_DIR = BASE_DIR+'/audio/united/'
WAV_DIR = BASE_DIR+'/audio/wavs/'
PARSED_DIR = BASE_DIR+'/parsed/united/'
SPECTRS_DIR = BASE_DIR+'/spectrs/'
CHORDS_DIR = BASE_DIR+'/chords/'

In [3]:
songs_ds_fname = DSETS_DIR+'united_songs_ds.csv'
converted_ds_fname = DSETS_DIR+'converted_ds.csv'

# Создаём датасет, для информации об изменении форматов форматов

In [4]:
#conv_ds = pd.read_csv(songs_ds_fname,index_col=0)
#conv_ds['wav'] = 'not tried'
#conv_ds['npz'] = 'not tried'
#conv_ds['chords'] = 'not tried'
#conv_ds.to_csv(converted_ds_fname)

# Mp3 To Wav

Загружаем датасет песен

In [5]:
conv_ds = pd.read_csv(converted_ds_fname,index_col=0)
conv_ds.head()

,title,artist,number,duration,t_eps,href,ds,wav,npz,chords
0,i_dont_mind,james_brown,3,02:31,1.0,NaN,billboard,success,not tried,not tried
1,youve_got_a_friend,roberta_flack_and_donny_hathaway,4,03:27,0.0,NaN,billboard,success,not tried,not tried
2,the_rose,bette_midler,6,03:41,-1.0,NaN,billboard,success,not tried,not tried
3,an_innocent_man,billy_joel,10,05:18,0.0,NaN,billboard,success,not tried,not tried
4,lookin_for_love,johnny_lee,12,03:32,0.0,NaN,billboard,success,not tried,not tried


Смотрим, какие песни уже сконвертированы в wav

In [6]:
conv_ds['wav'] = 'not tried'
for row in tqdm_notebook(conv_ds.iterrows(), total=len(conv_ds), desc='Walking rows'):
    idx, row = row
    
    prename = 'bb' if row['ds']=='billboard' else 'cl'
    fname = '{}_{}_{}-{}'.format(prename, row['number'], row['artist'], row['title'])
    if os.path.isfile(WAV_DIR+fname+'.wav'):
        conv_ds.at[idx,'wav'] = 'success'
else:
    conv_ds.to_csv(converted_ds_fname)

A Jupyter Widget

Выносим непереведённые песни в отдельный датасет

In [7]:
not_conv_ds = conv_ds[conv_ds['wav']!='success']
not_conv_ds.shape

(5, 10)

In [8]:
%%time

for row in tqdm_notebook(not_conv_ds.iterrows(), total=len(not_conv_ds), desc='Walking rows'):
    idx, row = row
    if idx%5==0: conv_ds.to_csv(converted_ds_fname)
    
    prename = 'bb' if row['ds']=='billboard' else 'cl'
    fname = '{}_{}_{}-{}'.format(prename, row['number'], row['artist'], row['title'])
    try:
        sound = AudioSegment.from_mp3(MP3_DIR+fname+'.mp3')
        mono = sound.set_channels(1)
        mono.export(WAV_DIR+fname+'.wav', format="wav")
        conv_ds.at[idx, 'wav'] = 'success'
    except Exception as e:
        conv_ds.at[idx, 'wav'] = str(e)
else:
    conv_ds.to_csv(converted_ds_fname)        

A Jupyter Widget


CPU times: user 76 ms, sys: 20 ms, total: 96 ms
Wall time: 238 ms


In [9]:
succ_num = len(conv_ds[conv_ds['wav']=='success'])
ntried_num = len(conv_ds[conv_ds['wav']=='not tried'])
error_num = len(conv_ds)-succ_num-ntried_num
print('Успешно переведено в wav:\t{}\t{:.2%}'.format(succ_num, succ_num/len(conv_ds)))
print('Ошибочно переведено в wav:\t{}\t{:.2%}'.format(error_num, error_num/len(conv_ds)))
print('Не переводилось в wav:\t\t{}\t{:.2%}'.format(ntried_num, ntried_num/len(conv_ds)))

Успешно переведено в wav:	1024	99.51%
Ошибочно переведено в wav:	5	0.49%
Не переводилось в wav:		0	0.00%


# Wav To Npz

## Тестируем перевод для одного файла

Значения установлены, согласно работам А. Кольяти (A. Cogliati)

In [10]:
flength = 2048
wind = signal.hamming
hopsize = 128

Параметры функции stft.spectrogram по умолчанию:

stft.spectrogram(data, framelength=1024, hopsize=None, overlap=None, centered=True, window=None, halved=True, transform=None, padding=0, save_settings=True)

Размер спектрограммы == частоты x временные_кадры

Переводим файл в тестовом режиме

In [11]:
sample_rate, samples = wavfile.read(WAV_DIR+'bb_3_james_brown-i_dont_mind.wav')
spectrogramm = stft.spectrogram(samples, framelength=flength, hopsize=hopsize, window=wind)
spectrogramm = absolute(spectrogramm)
spectrogramm.shape

(1025, 51777)

Спектрограмма получается невероятно большой и тяжёлой для памяти. Попробуем преобразовать её в разреженный вид

In [12]:
test_limit = 7000
n_small = int((spectrogramm<test_limit).sum())
print('Максимальное значение пустой ячейки:\t{}'.format(test_limit))
print('Кол-во пустых ячеек:\t{}\t{:.2%}'.format(n_small, n_small/spectrogramm.size))

Максимальное значение пустой ячейки:	7000
Кол-во пустых ячеек:	47830625	90.13%


Значение лимита выбрано из-за красивого числа 90%. Его, конечно, надо теоретически обосновать, но это позже.

In [13]:
spectrogramm[spectrogramm<test_limit]=0
spar = csc_matrix(spectrogramm)

In [14]:
del spectrogramm
del spar

## Переводим все файлы

Загружаем датасет

In [15]:
conv_ds = pd.read_csv(converted_ds_fname, index_col=0)
conv_ds.head()

,title,artist,number,duration,t_eps,href,ds,wav,npz,chords
0,i_dont_mind,james_brown,3,02:31,1.0,NaN,billboard,success,not tried,not tried
1,youve_got_a_friend,roberta_flack_and_donny_hathaway,4,03:27,0.0,NaN,billboard,success,not tried,not tried
2,the_rose,bette_midler,6,03:41,-1.0,NaN,billboard,success,not tried,not tried
3,an_innocent_man,billy_joel,10,05:18,0.0,NaN,billboard,success,not tried,not tried
4,lookin_for_love,johnny_lee,12,03:32,0.0,NaN,billboard,success,not tried,not tried


Проверяем уже переведённые файлы

In [16]:
wav_success_ds = conv_ds[conv_ds['wav']=='success']
wav_success_ds.shape

(1024, 10)

In [17]:
conv_ds['npz'] = 'not tried'
for row in tqdm_notebook(wav_success_ds.iterrows(), total=len(wav_success_ds), desc='Walking rows'):
    idx, row = row
    
    prename = 'bb' if row['ds']=='billboard' else 'cl'
    fname = '{}_{}_{}-{}'.format(prename, row['number'], row['artist'], row['title'])
    
    if os.path.isfile(SPECTRS_DIR+fname+'.npz'):
        conv_ds.at[idx,'npz'] = 'success'

A Jupyter Widget

Выносим непереведённые файлы в отдельный датасет

In [18]:
not_conv_ds = conv_ds[(conv_ds['wav']=='success')&(conv_ds['npz']!='success')]
not_conv_ds.shape

(1017, 10)

Пороговое значение. Все ячейки спектрограммы ниже него, приравниваются к нулю

*!!!Внимание!!!* Установлен (почти) случайным образом

In [19]:
limit = 7000

Переводим файлы

In [20]:
%%time

for row in tqdm_notebook(not_conv_ds.iterrows(), total=len(not_conv_ds), desc='Walking rows'):
    idx, row = row
    if idx%5==0: conv_ds.to_csv(converted_ds_fname)
    
    prename = 'bb' if row['ds']=='billboard' else 'cl'
    fname = '{}_{}_{}-{}'.format(prename, row['number'], row['artist'], row['title'])
    
    try:
        sample_rate, samples = wavfile.read(WAV_DIR+fname+'.wav')
        spm = absolute(stft.spectrogram(samples, framelength=flength, hopsize=hopsize, window=wind))

        spm[spm<limit]=0
        spar = csc_matrix(spm)
        save_npz(SPECTRS_DIR+fname+'.npz', spar)
        conv_ds.at[idx,'npz'] = 'success'
    except Exception as e:
        conv_ds.at[idx,'npz'] = str(e)
else:
    conv_ds.to_csv(converted_ds_fname)

A Jupyter Widget


CPU times: user 6h 32min 27s, sys: 20min 38s, total: 6h 53min 6s
Wall time: 7h 14min 33s


In [21]:
succ_num = len(conv_ds[conv_ds['npz']=='success'])
ntried_num = len(conv_ds[conv_ds['npz']=='not tried'])
error_num = len(conv_ds)-succ_num-ntried_num
print('Успешно переведено в npz:\t{}\t{:.2%}'.format(succ_num, succ_num/len(conv_ds)))
print('Ошибочно переведено в npz:\t{}\t{:.2%}'.format(error_num, error_num/len(conv_ds)))
print('Не переводилось в npz:\t\t{}\t{:.2%}'.format(ntried_num, ntried_num/len(conv_ds)))

Успешно переведено в npz:	1024	99.51%
Ошибочно переведено в npz:	0	0.00%
Не переводилось в npz:		5	0.49%


# Npz To Chords

Загружаем датасет

In [22]:
conv_ds = pd.read_csv(converted_ds_fname, index_col=0)
conv_ds.head()

,title,artist,number,duration,t_eps,href,ds,wav,npz,chords
0,i_dont_mind,james_brown,3,02:31,1.0,NaN,billboard,success,success,not tried
1,youve_got_a_friend,roberta_flack_and_donny_hathaway,4,03:27,0.0,NaN,billboard,success,success,not tried
2,the_rose,bette_midler,6,03:41,-1.0,NaN,billboard,success,success,not tried
3,an_innocent_man,billy_joel,10,05:18,0.0,NaN,billboard,success,success,not tried
4,lookin_for_love,johnny_lee,12,03:32,0.0,NaN,billboard,success,success,not tried


Удаляем все созданные прежде спектрограммы аккордов. Это проще, чем считать, какие из них уже были вычленены из песен. Хотя потом и это можно сделать

In [23]:
shutil.rmtree(CHORDS_DIR, ignore_errors=True)
os.mkdir(CHORDS_DIR)

Вынесем неразделённые песни в отдельный датасет

In [67]:
no_chords_ds = conv_ds[(conv_ds['wav']=='success')&(conv_ds['npz']=='success')&(conv_ds['chords']!='success')]
no_chords_ds.shape

(176, 10)

В песнях The Beatles датасета chordlab, в отличие от остальных файлов, в качестве пробельных символов стоят пробелы, а не табуляции

In [66]:
for fname in os.listdir(PARSED_DIR):
    with open(PARSED_DIR+fname, 'r') as f:
        lines = f.readlines()
    lines = [l.replace(' ', '\t') for l in lines]
    with open(PARSED_DIR+fname, 'w') as f:
        f.write(''.join(lines))

In [ ]:
%%time

for song_row in tqdm_notebook(no_chords_ds.iterrows(), total=len(no_chords_ds), desc='Walking rows'):
    song_idx, song_row = song_row
    if song_idx%5==0: conv_ds.to_csv(converted_ds_fname)
    
    try:
        dur = string_to_secs(song_row['duration'])+song_row['t_eps']
        prename = 'bb' if song_row['ds']=='billboard' else 'cl'

        fname = '{}_{}_{}-{}'.format(prename, song_row['number'], song_row['artist'], song_row['title'])
        song_df = pd.read_csv(PARSED_DIR+fname+'.lab', delimiter='\t',header=None, names=['start','stop','label'])    
        full_sp = load_npz(SPECTRS_DIR+fname+'.npz')
        sp_block_time = dur/full_sp.shape[1]

        chords_sp = {}
        for ch_row in tqdm_notebook(song_df.iterrows(), total=len(song_df), desc='Walking chord', leave=False):
            ch_idx, ch_row = ch_row
            start, stop = list(map(lambda x: round( round(float(x)/sp_block_time)*sp_block_time ),
                                  [ch_row['start'],ch_row['stop']]))
            chords_sp[ch_row['label']]= hstack([chords_sp.get(ch_row['label']),
                                                         full_sp[:,start:stop]])

        for chName, chSp in chords_sp.items():
            chName = chName.replace(r'/', '_inv_')
            if not os.path.isdir(CHORDS_DIR+chName):
                os.mkdir(CHORDS_DIR+chName)
            save_npz(CHORDS_DIR+chName+'/'+fname+'.npz',chSp)
        conv_ds.at[song_idx,'chords'] = 'success'
    except Exception as e:
        conv_ds.at[song_idx,'chords'] = str(e)
else:
    if song_idx%5==0: conv_ds.to_csv(converted_ds_fname)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

...

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [ ]:
succ_num = len(conv_ds[conv_ds['chords']=='success'])
ntried_num = len(conv_ds[conv_ds['chords']=='not tried'])
error_num = len(conv_ds)-succ_num-ntried_num
print('Успешно переведено в npz:\t{}\t{:.2%}'.format(succ_num, succ_num/len(conv_ds)))
print('Ошибочно переведено в npz:\t{}\t{:.2%}'.format(error_num, error_num/len(conv_ds)))
print('Не переводилось в npz:\t\t{}\t{:.2%}'.format(ntried_num, ntried_num/len(conv_ds)))